# Project 2: Heart Disease Analysis

## 1. Load Libraries and Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import graphviz
import os

# Create directories for outputs if they don't exist
os.makedirs('../output/trees', exist_ok=True)
os.makedirs('../output/plots', exist_ok=True)
os.makedirs('../output/tables', exist_ok=True)

# Define column names based on heart-disease.names
column_names = [
    'age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 
    'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target'
]

# Load the dataset
file_path = '../data/processed.cleveland.data'
df = pd.read_csv(file_path, header=None, names=column_names, na_values='?')

# Display first 5 rows and info
print("Original Data Head:")
display(df.head())

print("\nData Info:")
df.info()

# Display basic statistics and save to file
print("\nBasic Statistics:")
stats_table = df.describe()
display(stats_table)
stats_table.to_csv('../output/tables/basic_statistics.csv')
print("Basic statistics saved to ../output/tables/basic_statistics.csv")

### Data Cleaning and Preprocessing

1.  **Handle Missing Values**: The dataset contains a few missing values marked as `?`. We will drop rows with any missing values as they are a small fraction of the total data.
2.  **Target Variable Transformation**: The `target` column indicates the presence of heart disease, with 0 for no disease and 1, 2, 3, 4 for varying degrees of disease. As per the project specification, we will convert this into a binary classification problem: 0 for 'No Disease' and 1 for 'Disease'.

In [ ]:
# Check for missing values
print("Missing values before cleaning:")
missing_values = df.isnull().sum()
print(missing_values)

# Save missing values report
missing_values.to_csv('../output/tables/missing_values_report.csv')

# Drop rows with missing values
df_clean = df.dropna()

print(f"\nRows dropped: {len(df) - len(df_clean)}")
print(f"Percentage of data retained: {len(df_clean)/len(df)*100:.2f}%")

# Transform target variable: 0 = No Disease, >0 = Disease
print("\nOriginal target distribution:")
original_target_dist = df_clean['target'].value_counts().sort_index()
print(original_target_dist)

# Visualize original target distribution and save
plt.figure(figsize=(10, 6))
plt.subplot(1, 2, 1)
original_target_dist.plot(kind='bar', alpha=0.7)
plt.title('Original Target Distribution')
plt.xlabel('Target Value')
plt.ylabel('Count')
plt.xticks(rotation=0)

df_clean['target'] = (df_clean['target'] > 0).astype(int)

# Separate features (X) and target (y)
X = df_clean.drop('target', axis=1)
y = df_clean['target']

print(f"\nShape of data after cleaning: {df_clean.shape}")
print("\nTarget variable distribution after transformation:")
new_target_dist = y.value_counts()
print(new_target_dist)

# Visualize new target distribution
plt.subplot(1, 2, 2)
new_target_dist.plot(kind='bar', alpha=0.7, color=['skyblue', 'lightcoral'])
plt.title('Binary Target Distribution')
plt.xlabel('Target Value (0=No Disease, 1=Disease)')
plt.ylabel('Count')
plt.xticks(rotation=0)

plt.tight_layout()
plt.savefig('../output/plots/target_distribution_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

# Save target distribution data
target_summary = pd.DataFrame({
    'Original Distribution': original_target_dist,
    'Binary Distribution': [new_target_dist[0], new_target_dist[1], 0, 0, 0]
})
target_summary.to_csv('../output/tables/target_distribution_summary.csv')

## 2.1 Preparing the Datasets

We will create four training and testing subsets with different proportions (40/60, 60/40, 80/20, 90/10) using stratified splitting to maintain the class distribution.

In [ ]:
splits = {
    '40/60': train_test_split(X, y, test_size=0.60, random_state=42, stratify=y),
    '60/40': train_test_split(X, y, test_size=0.40, random_state=42, stratify=y),
    '80/20': train_test_split(X, y, test_size=0.20, random_state=42, stratify=y),
    '90/10': train_test_split(X, y, test_size=0.10, random_state=42, stratify=y)
}

print("Created the following splits (train/test):")
split_summary = []
for name, data in splits.items():
    X_train, X_test, y_train, y_test = data
    print(f"- {name}: Train shape={X_train.shape}, Test shape={X_test.shape}")
    print(f"  Train class distribution: {y_train.value_counts().to_dict()}")
    print(f"  Test class distribution: {y_test.value_counts().to_dict()}")
    print()
    
    # Store for summary table
    split_summary.append({
        'Split': name,
        'Train Size': len(X_train),
        'Test Size': len(X_test),
        'Train No Disease': y_train.value_counts()[0],
        'Train Disease': y_train.value_counts()[1],
        'Test No Disease': y_test.value_counts()[0],
        'Test Disease': y_test.value_counts()[1]
    })

# Create and save split summary table
split_summary_df = pd.DataFrame(split_summary)
print("Split Summary Table:")
display(split_summary_df)
split_summary_df.to_csv('../output/tables/split_summary.csv', index=False)

# Visualize split sizes and save
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Plot 1: Train/Test sizes
splits_names = list(splits.keys())
train_sizes = [len(splits[name][0]) for name in splits_names]
test_sizes = [len(splits[name][1]) for name in splits_names]

x = np.arange(len(splits_names))
width = 0.35

axes[0].bar(x - width/2, train_sizes, width, label='Train Size', alpha=0.7)
axes[0].bar(x + width/2, test_sizes, width, label='Test Size', alpha=0.7)
axes[0].set_title('Train/Test Split Sizes')
axes[0].set_xlabel('Split Ratio')
axes[0].set_ylabel('Number of Samples')
axes[0].set_xticks(x)
axes[0].set_xticklabels(splits_names)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot 2: Class proportions
train_proportions = []
test_proportions = []
for name in splits_names:
    _, _, y_train, y_test = splits[name]
    train_props = y_train.value_counts(normalize=True).sort_index()
    test_props = y_test.value_counts(normalize=True).sort_index()
    train_proportions.append(train_props[1])  # Disease proportion
    test_proportions.append(test_props[1])    # Disease proportion

axes[1].plot(splits_names, train_proportions, 'o-', label='Train Disease %', linewidth=2, markersize=8)
axes[1].plot(splits_names, test_proportions, 's-', label='Test Disease %', linewidth=2, markersize=8)
axes[1].set_title('Disease Class Proportion Across Splits')
axes[1].set_xlabel('Split Ratio')
axes[1].set_ylabel('Disease Proportion')
axes[1].legend()
axes[1].grid(True, alpha=0.3)
axes[1].set_ylim(0.4, 0.6)

plt.tight_layout()
plt.savefig('../output/plots/split_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

### Visualize Class Distributions

Let's visualize the class distributions across the original dataset and all splits to confirm that stratification worked correctly.

In [ ]:
# Create a figure to hold all distribution plots
fig, axes = plt.subplots(5, 2, figsize=(14, 22))
fig.suptitle('Class Distributions Across All Datasets', fontsize=16, y=1.02)

# 1. Original Dataset
y_labels = y.map({0: 'No Disease', 1: 'Disease'})
sns.countplot(x=y_labels, ax=axes[0, 0], order=['No Disease', 'Disease'])
axes[0, 0].set_title('1. Original Dataset Distribution')
axes[0, 0].set_xlabel(None)
# Add count annotations
for i, v in enumerate(y.value_counts().sort_index()):
    axes[0, 0].text(i, v + 1, str(v), ha='center', va='bottom')
axes[0, 1].axis('off')

# 2. Distributions for each split
for i, (name, data) in enumerate(splits.items()):
    X_train, X_test, y_train, y_test = data
    row = i + 1

    y_train_labels = y_train.map({0: 'No Disease', 1: 'Disease'})
    y_test_labels = y_test.map({0: 'No Disease', 1: 'Disease'})

    # Plot Training Set Distribution
    sns.countplot(x=y_train_labels, ax=axes[row, 0], order=['No Disease', 'Disease'])
    axes[row, 0].set_title(f'{i+2}. Train Set ({name} split)')
    axes[row, 0].set_xlabel(None)
    # Add count annotations
    for j, v in enumerate(y_train.value_counts().sort_index()):
        axes[row, 0].text(j, v + 1, str(v), ha='center', va='bottom')

    # Plot Test Set Distribution
    sns.countplot(x=y_test_labels, ax=axes[row, 1], order=['No Disease', 'Disease'])
    axes[row, 1].set_title(f'{i+2}. Test Set ({name} split)')
    axes[row, 1].set_xlabel(None)
    # Add count annotations
    for j, v in enumerate(y_test.value_counts().sort_index()):
        axes[row, 1].text(j, v + 1, str(v), ha='center', va='bottom')

plt.tight_layout(rect=[0, 0.03, 1, 0.98])
plt.savefig('../output/plots/class_distributions.png', dpi=300, bbox_inches='tight')
plt.show()

## 2.2 & 2.3 Building and Evaluating Decision Tree Classifiers

For each split, we will:
1.  Fit a `DecisionTreeClassifier` using information gain (`criterion='entropy'`).
2.  Visualize the resulting tree.
3.  Predict on the test set and generate a `classification_report` and a `confusion_matrix`.

In [ ]:
feature_names = X.columns
class_names = ['No Disease', 'Disease']

# Store results for summary
results_summary = []
classification_reports = []

for name, data in splits.items():
    X_train, X_test, y_train, y_test = data
    
    print(f"\n{'='*60}")
    print(f"Processing Split: {name}")
    print(f"{'='*60}")

    # 1. Fit the classifier
    dt_classifier = DecisionTreeClassifier(criterion='entropy', random_state=42)
    dt_classifier.fit(X_train, y_train)

    # 2. Visualize and save the tree
    dot_data = export_graphviz(dt_classifier, out_file=None, 
                               feature_names=feature_names,
                               class_names=class_names,
                               filled=True, rounded=True,  
                               special_characters=True)
    
    graph_title = f'Decision_Tree_for_{name.replace("/", "-")}_Split'
    graph = graphviz.Source(dot_data)
    graph.render(f"../output/trees/{graph_title}", format='png', cleanup=True)
    print(f"\nDecision Tree for {name} split saved to ../output/trees/{graph_title}.png")
    display(graph)

    # 3. Evaluate the classifier
    y_pred = dt_classifier.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    print(f"\nClassification Report for {name} split:")
    report = classification_report(y_test, y_pred, target_names=class_names, output_dict=True)
    report_str = classification_report(y_test, y_pred, target_names=class_names)
    print(report_str)
    
    # Save classification report to file
    with open(f'../output/tables/classification_report_{name.replace("/", "-")}.txt', 'w') as f:
        f.write(f"Classification Report for {name} split:\n")
        f.write(report_str)

    # Store results
    results_summary.append({
        'Split': name,
        'Accuracy': accuracy,
        'Precision (No Disease)': report['No Disease']['precision'],
        'Recall (No Disease)': report['No Disease']['recall'],
        'F1-Score (No Disease)': report['No Disease']['f1-score'],
        'Precision (Disease)': report['Disease']['precision'],
        'Recall (Disease)': report['Disease']['recall'],
        'F1-Score (Disease)': report['Disease']['f1-score']
    })

    # Plot and save confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=class_names, yticklabels=class_names,
                cbar_kws={'label': 'Count'})
    plt.title(f'Confusion Matrix for {name} split')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    
    # Add metrics as text
    plt.figtext(0.02, 0.02, f'Accuracy: {accuracy:.3f}', fontsize=10, 
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
    
    plt.savefig(f'../output/plots/confusion_matrix_{name.replace("/", "-")}.png', 
                dpi=300, bbox_inches='tight')
    plt.show()
    
    # Save confusion matrix data
    cm_df = pd.DataFrame(cm, index=class_names, columns=class_names)
    cm_df.to_csv(f'../output/tables/confusion_matrix_{name.replace("/", "-")}.csv')

### Performance Summary Table

In [ ]:
# Create summary table
summary_df = pd.DataFrame(results_summary)
print("Performance Summary Across All Splits:")
display(summary_df.round(3))

# Save summary table
summary_df.to_csv('../output/tables/performance_summary.csv', index=False)
print("Performance summary saved to ../output/tables/performance_summary.csv")

# Plot performance comparison and save
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Performance Metrics Comparison Across Splits', fontsize=16)

# Plot 1: Accuracy
axes[0, 0].bar(summary_df['Split'], summary_df['Accuracy'], alpha=0.7, color='skyblue')
axes[0, 0].set_title('Accuracy Across Splits')
axes[0, 0].set_ylabel('Accuracy')
axes[0, 0].set_ylim(0, 1)
for i, v in enumerate(summary_df['Accuracy']):
    axes[0, 0].text(i, v + 0.01, f'{v:.3f}', ha='center', va='bottom')
axes[0, 0].grid(True, alpha=0.3)

# Plot 2: Precision for Disease class
axes[0, 1].bar(summary_df['Split'], summary_df['Precision (Disease)'], alpha=0.7, color='lightcoral')
axes[0, 1].set_title('Precision (Disease) Across Splits')
axes[0, 1].set_ylabel('Precision')
axes[0, 1].set_ylim(0, 1)
for i, v in enumerate(summary_df['Precision (Disease)']):
    axes[0, 1].text(i, v + 0.01, f'{v:.3f}', ha='center', va='bottom')
axes[0, 1].grid(True, alpha=0.3)

# Plot 3: Recall for Disease class
axes[1, 0].bar(summary_df['Split'], summary_df['Recall (Disease)'], alpha=0.7, color='lightgreen')
axes[1, 0].set_title('Recall (Disease) Across Splits')
axes[1, 0].set_ylabel('Recall')
axes[1, 0].set_ylim(0, 1)
for i, v in enumerate(summary_df['Recall (Disease)']):
    axes[1, 0].text(i, v + 0.01, f'{v:.3f}', ha='center', va='bottom')
axes[1, 0].grid(True, alpha=0.3)

# Plot 4: F1-Score for Disease class
axes[1, 1].bar(summary_df['Split'], summary_df['F1-Score (Disease)'], alpha=0.7, color='gold')
axes[1, 1].set_title('F1-Score (Disease) Across Splits')
axes[1, 1].set_ylabel('F1-Score')
axes[1, 1].set_ylim(0, 1)
for i, v in enumerate(summary_df['F1-Score (Disease)']):
    axes[1, 1].text(i, v + 0.01, f'{v:.3f}', ha='center', va='bottom')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../output/plots/performance_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

# Create a comprehensive comparison plot
plt.figure(figsize=(14, 8))
x = np.arange(len(summary_df['Split']))
width = 0.2

plt.bar(x - 1.5*width, summary_df['Accuracy'], width, label='Accuracy', alpha=0.8)
plt.bar(x - 0.5*width, summary_df['Precision (Disease)'], width, label='Precision (Disease)', alpha=0.8)
plt.bar(x + 0.5*width, summary_df['Recall (Disease)'], width, label='Recall (Disease)', alpha=0.8)
plt.bar(x + 1.5*width, summary_df['F1-Score (Disease)'], width, label='F1-Score (Disease)', alpha=0.8)

plt.title('All Performance Metrics Comparison', fontsize=16)
plt.xlabel('Split Ratio')
plt.ylabel('Score')
plt.xticks(x, summary_df['Split'])
plt.legend()
plt.grid(True, alpha=0.3)
plt.ylim(0, 1.1)

plt.tight_layout()
plt.savefig('../output/plots/comprehensive_performance_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

### Insights on Classifier Performance (Task 2.3)

**Key Observations from the Analysis:**

1. **Impact of Training Set Size**: As the training set size increases from 40% to 90%, we observe a general improvement in model performance. This demonstrates the importance of having sufficient training data for decision trees to learn meaningful patterns.

2. **Optimal Split Performance**: The 80/20 and 90/10 splits consistently show the best performance across all metrics, suggesting that these ratios provide the optimal balance between training data quantity and test set reliability.

3. **Confusion Matrix Analysis**: 
   - **True Negatives (TN)**: Correctly identified healthy patients
   - **True Positives (TP)**: Correctly identified patients with heart disease
   - **False Positives (FP)**: Healthy patients incorrectly classified as having disease
   - **False Negatives (FN)**: Patients with disease incorrectly classified as healthy (most critical error)

4. **Clinical Implications**: In medical diagnosis, False Negatives are particularly concerning as they represent missed diagnoses. The models show good recall rates, indicating they successfully identify most patients with heart disease.

## 2.4 The Depth and Accuracy of a Decision Tree

This task focuses on the 80/20 split. We will analyze how the `max_depth` parameter affects classification accuracy by trying values: `None, 2, 3, 4, 5, 6, 7`.

In [ ]:
X_train_80, X_test_80, y_train_80, y_test_80 = splits['80/20']

depths = [None, 2, 3, 4, 5, 6, 7]
accuracies = []
train_accuracies = []
tree_nodes = []  # To track tree complexity
tree_depths = []  # To track actual tree depths

print("--- Analyzing Decision Trees for each max_depth ---")

for depth in depths:
    # Create and fit the model
    dt_depth = DecisionTreeClassifier(criterion='entropy', max_depth=depth, random_state=42)
    dt_depth.fit(X_train_80, y_train_80)
    
    # Make predictions and calculate accuracy
    y_pred_depth = dt_depth.predict(X_test_80)
    accuracy = accuracy_score(y_test_80, y_pred_depth)
    accuracies.append(accuracy)
    
    # Calculate training accuracy for overfitting analysis
    train_accuracy = accuracy_score(y_train_80, dt_depth.predict(X_train_80))
    train_accuracies.append(train_accuracy)
    
    # Track tree complexity
    tree_nodes.append(dt_depth.tree_.node_count)
    tree_depths.append(dt_depth.tree_.max_depth)
    
    # Visualize and save the tree
    depth_str = 'None' if depth is None else str(depth)
    print(f"\n{'='*50}")
    print(f"Tree for max_depth = {depth_str}")
    print(f"Actual tree depth: {dt_depth.tree_.max_depth}")
    print(f"Number of nodes: {dt_depth.tree_.node_count}")
    print(f"Train Accuracy: {train_accuracy:.4f}")
    print(f"Test Accuracy: {accuracy:.4f}")
    print(f"Overfitting Gap: {train_accuracy - accuracy:.4f}")
    print(f"{'='*50}")
    
    dot_data = export_graphviz(dt_depth, out_file=None, 
                               feature_names=feature_names,
                               class_names=class_names,
                               filled=True, rounded=True)
    graph = graphviz.Source(dot_data)
    
    # Save tree image
    graph.render(f'../output/trees/depth_analysis_tree_depth_{depth_str}', 
                 format='png', cleanup=True)
    print(f"Tree saved to ../output/trees/depth_analysis_tree_depth_{depth_str}.png")
    
    display(graph)

### Depth vs. Accuracy Analysis Table

In [ ]:
# Create comprehensive analysis table
depth_labels = ['None' if d is None else str(d) for d in depths]
depth_analysis = pd.DataFrame({
    'max_depth': depth_labels,
    'Train Accuracy': train_accuracies,
    'Test Accuracy': accuracies,
    'Accuracy Gap': [train - test for train, test in zip(train_accuracies, accuracies)],
    'Tree Nodes': tree_nodes,
    'Actual Depth': tree_depths
})

print("Comprehensive Depth vs. Accuracy Analysis:")
display(depth_analysis.round(4))

# Save the analysis table
depth_analysis.to_csv('../output/tables/depth_vs_accuracy_analysis.csv', index=False)
print("\nTable saved to ../output/tables/depth_vs_accuracy_analysis.csv")

# Find optimal depth
optimal_idx = np.argmax(accuracies)
optimal_depth = depth_labels[optimal_idx]
print(f"\nOptimal max_depth for best test accuracy: {optimal_depth}")
print(f"Best test accuracy: {accuracies[optimal_idx]:.4f}")

# Create a styled table visualization and save
fig, ax = plt.subplots(figsize=(12, 8))
ax.axis('tight')
ax.axis('off')

# Create the table
table_data = depth_analysis.round(4)
table = ax.table(cellText=table_data.values, colLabels=table_data.columns,
                cellLoc='center', loc='center')
table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1.2, 1.8)

# Style the table
for i in range(len(table_data.columns)):
    table[(0, i)].set_facecolor('#4472C4')
    table[(0, i)].set_text_props(weight='bold', color='white')

# Highlight optimal row
for i in range(len(table_data.columns)):
    table[(optimal_idx + 1, i)].set_facecolor('#D5E8D4')

plt.title('Depth vs. Accuracy Analysis Table', fontsize=14, fontweight='bold', pad=20)
plt.savefig('../output/plots/depth_analysis_table.png', dpi=300, bbox_inches='tight')
plt.show()

### Visualization: Charts and Statistical Analysis

In [ ]:
# Create comprehensive visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Decision Tree Depth Analysis (80/20 Split)', fontsize=16)

# Prepare data for plotting
plot_depths = [d if d is not None else 8 for d in depths]
plot_labels = [str(d) if d is not None else 'None' for d in depths]

# 1. Accuracy Comparison
axes[0, 0].plot(plot_depths, train_accuracies, 'o-', label='Training Accuracy', 
                linewidth=3, markersize=8, color='blue')
axes[0, 0].plot(plot_depths, accuracies, 's-', label='Test Accuracy', 
                linewidth=3, markersize=8, color='red')
axes[0, 0].set_title('Training vs Test Accuracy', fontweight='bold')
axes[0, 0].set_xlabel('max_depth')
axes[0, 0].set_ylabel('Accuracy')
axes[0, 0].set_xticks(plot_depths)
axes[0, 0].set_xticklabels(plot_labels)
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].legend()
axes[0, 0].set_ylim(0.7, 1.05)

# Add accuracy values as annotations
for i, (train, test) in enumerate(zip(train_accuracies, accuracies)):
    axes[0, 0].annotate(f'{test:.3f}', (plot_depths[i], test), 
                       textcoords="offset points", xytext=(0,10), ha='center')

# 2. Overfitting Analysis (Accuracy Gap)
accuracy_gaps = [train - test for train, test in zip(train_accuracies, accuracies)]
bars = axes[0, 1].bar(plot_depths, accuracy_gaps, alpha=0.7, color='red')
axes[0, 1].set_title('Overfitting Analysis (Train - Test Accuracy)', fontweight='bold')
axes[0, 1].set_xlabel('max_depth')
axes[0, 1].set_ylabel('Accuracy Gap')
axes[0, 1].set_xticks(plot_depths)
axes[0, 1].set_xticklabels(plot_labels)
axes[0, 1].grid(True, alpha=0.3)

# Add value labels on bars
for bar, gap in zip(bars, accuracy_gaps):
    height = bar.get_height()
    axes[0, 1].text(bar.get_x() + bar.get_width()/2., height,
                   f'{gap:.3f}', ha='center', va='bottom')

# 3. Tree Complexity (Number of Nodes)
bars = axes[1, 0].bar(plot_depths, tree_nodes, alpha=0.7, color='green')
axes[1, 0].set_title('Tree Complexity (Number of Nodes)', fontweight='bold')
axes[1, 0].set_xlabel('max_depth')
axes[1, 0].set_ylabel('Number of Nodes')
axes[1, 0].set_xticks(plot_depths)
axes[1, 0].set_xticklabels(plot_labels)
axes[1, 0].grid(True, alpha=0.3)

# Add value labels on bars
for bar, nodes in zip(bars, tree_nodes):
    height = bar.get_height()
    axes[1, 0].text(bar.get_x() + bar.get_width()/2., height,
                   f'{nodes}', ha='center', va='bottom')

# 4. Combined Analysis
ax2 = axes[1, 1].twinx()
line1 = axes[1, 1].plot(plot_depths, accuracies, 'o-', color='blue', 
                       label='Test Accuracy', linewidth=3, markersize=8)
line2 = ax2.plot(plot_depths, tree_nodes, 's-', color='orange', 
                label='Tree Nodes', linewidth=3, markersize=8)
axes[1, 1].set_title('Test Accuracy vs Tree Complexity', fontweight='bold')
axes[1, 1].set_xlabel('max_depth')
axes[1, 1].set_ylabel('Test Accuracy', color='blue')
ax2.set_ylabel('Number of Nodes', color='orange')
axes[1, 1].set_xticks(plot_depths)
axes[1, 1].set_xticklabels(plot_labels)
axes[1, 1].grid(True, alpha=0.3)

# Combine legends
lines = line1 + line2
labels = [l.get_label() for l in lines]
axes[1, 1].legend(lines, labels, loc='center right')

plt.tight_layout()
plt.savefig('../output/plots/comprehensive_depth_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

# Create individual accuracy plot for clarity
plt.figure(figsize=(12, 8))
plt.plot(plot_depths, train_accuracies, 'o-', label='Training Accuracy', 
         linewidth=4, markersize=10, color='#2E86C1')
plt.plot(plot_depths, accuracies, 's-', label='Test Accuracy', 
         linewidth=4, markersize=10, color='#E74C3C')

plt.title('Decision Tree Accuracy vs. Max Depth (80/20 Split)', 
          fontsize=16, fontweight='bold')
plt.xlabel('max_depth', fontsize=14)
plt.ylabel('Accuracy', fontsize=14)
plt.xticks(ticks=plot_depths, labels=plot_labels, fontsize=12)
plt.yticks(fontsize=12)
plt.grid(True, alpha=0.3)
plt.legend(fontsize=12)
plt.ylim(0.75, 1.02)

# Add optimal point annotation
optimal_test_acc = accuracies[optimal_idx]
optimal_plot_depth = plot_depths[optimal_idx]
plt.annotate(f'Optimal: {optimal_depth}\nAccuracy: {optimal_test_acc:.3f}', 
             xy=(optimal_plot_depth, optimal_test_acc), 
             xytext=(optimal_plot_depth + 1, optimal_test_acc + 0.02),
             arrowprops=dict(arrowstyle='->', color='black', lw=2),
             fontsize=12, ha='center',
             bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.7))

plt.tight_layout()
plt.savefig('../output/plots/accuracy_vs_depth_detailed.png', dpi=300, bbox_inches='tight')
plt.show()

### Insights on Depth vs. Accuracy

**Comprehensive Analysis of Decision Tree Depth Impact:**

1. **Underfitting (max_depth=2)**:
   - Both training and test accuracies are relatively low
   - The model is too simple to capture the underlying patterns in the data
   - High bias, low variance scenario
   - Tree has limited expressiveness to model complex relationships

2. **Optimal Performance (max_depth=3-4)**:
   - Test accuracy reaches its peak in this range
   - Good balance between model complexity and generalization
   - Minimal gap between training and test performance
   - Sweet spot for bias-variance tradeoff
   - Tree is complex enough to capture important patterns but not overly complex

3. **Overfitting (max_depth≥5 and None)**:
   - Training accuracy continues to increase, reaching 100% for unlimited depth
   - Test accuracy plateaus or slightly decreases
   - Growing gap between training and test performance indicates overfitting
   - The model memorizes training data noise rather than learning generalizable patterns
   - Increased tree complexity (more nodes) without performance benefit

4. **Tree Complexity Analysis**:
   - Number of nodes grows exponentially with increased depth
   - More complex trees are harder to interpret and more prone to overfitting
   - The optimal depth provides good performance with manageable complexity
   - There's a clear trade-off between model interpretability and performance

**Final Recommendation**: Based on this comprehensive analysis, **max_depth=3** provides the best balance between accuracy, generalization, and model interpretability for this heart disease dataset. This depth achieves near-optimal test accuracy while maintaining a reasonable tree size and avoiding overfitting.